# TAL aplicado al análisis del discurso de los medios de prensa 📰🤓🔥


El proyecto consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia.


- Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline

- Hito Unidad 2 (27 de octubre): Implementación y experimentos de varios modelos de clasificación

- Hito Proyecto (15 de diciembre): Evaluación y comparación de los modelos de los distintos equipos + integración de los mejores modelos en la arquitectura Sophia2.


### índex

1. [Importación del dataset](a)
2. [Extracción de tópicos vía RE en URLs](b)
3. [Extracción de términos clave de columnas title+text](b)
4. [Clasificación del dataset de entrenamiento](c)

## Generando dataset de entrenamiento
El dataset se clasificará en función de las siguientes etiquetas:
- mundo = 0 
- economía = 1 
- política y conflictos = 2 
- ciencia y tecnología = 3
- catástrofes y accidentes = 4 
- cultura y arte = 5 
- deporte = 6 
- ecología y planeta = 7
- crimen, delitos y justicia = 8 
- salud = 9

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import re
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# NLP
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
#import pyLDAvis
#import pyLDAvis.gensim_models  # don't skip this
import nltk 
#nltk.download('stopwords') 
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
print("versión de Spacy: ", spacy.__version__)

versión de Spacy:  3.1.3


### 1. Importación de dataset <a name="a"></a>

In [2]:
frames = [pd.read_csv("data/chile_2020-09.csv"), pd.read_csv("data/chile_2020-10.csv"),pd.read_csv("data/chile_2020-11.csv"), pd.read_csv("data/chile_2020-12.csv"), pd.read_csv("data/chile_2021-01.csv"), pd.read_csv("data/chile_2021-02.csv"), pd.read_csv("data/chile_2021-03.csv"), pd.read_csv("data/chile_2021-04.csv")]
dataset = pd.concat(frames)
#dataset = dataset[:2000]  #para reducir el coste computacional en desarrollo
dataset['content'] = dataset['title'] + " " + dataset['text'] 

print("Largo de dataset: ", len(dataset))
dataset= dataset[dataset['text'].notna()]
        
dataset.drop(columns=['country','year','date','id', 'id_journalist','title'], inplace=True)
dataset['topic'] = ""

Largo de dataset:  69702


### 2.  Extracción de tópicos vía RE en URL <a name="b"></a>

In [3]:
#identicando fuentes fuentes
set(dataset["media_outlet"])

{'ahoranoticiasmega',
 'biobiochile',
 'elciudadano',
 'elmostrador',
 'emol',
 'horas24',
 'latercera'}

In [4]:
print(dataset[dataset.media_outlet=="ahoranoticiasmega"]["url"].head(2))
print(dataset[dataset.media_outlet=="horas24"]["url"].head(2))
print(dataset[dataset.media_outlet=="elciudadano"]["url"].head(2))
print(dataset[dataset.media_outlet=="biobiochile"]["url"].head(2))
print(dataset[dataset.media_outlet=="elmostrador"]["url"].head(2))
print(dataset[dataset.media_outlet=="emol"]["url"].head(2))
print(dataset[dataset.media_outlet=="latercera"]["url"].head(2))

268    https://www.meganoticias.cl/nacional/312353-si...
269    https://www.meganoticias.cl/nacional/312350-cy...
Name: url, dtype: object
0    https://www.24horas.cl/coronavirus/tia-pikachu...
3    https://www.24horas.cl/nacional/rm-alcanza-sus...
Name: url, dtype: object
205    https://www.elciudadano.com/chile/el-avance-de...
206    https://www.elciudadano.com/latinoamerica/colo...
Name: url, dtype: object
29    https://www.biobiochile.cl/noticias/nacional/c...
30    https://www.biobiochile.cl/noticias/nacional/r...
Name: url, dtype: object
228    https://www.elmostrador.cl/noticias/pais/2020/...
230    https://www.elmostrador.cl/dia/2020/09/01/dipu...
Name: url, dtype: object
496    https://www.emol.com/noticias/Economia/2020/09...
497    https://www.emol.com/noticias/Economia/2020/09...
Name: url, dtype: object
209    https://www.latercera.com/politica/noticia/el-...
210    https://www.latercera.com/politica/noticia/cam...
Name: url, dtype: object


Las urls de _Ahoranoticiasmega, Horas24 y El Ciudadano_ son similares.
La url de _La Tercera_ posee dos tópicos, sin embargo, el segundo tópico es irrelevante.

In [5]:
#Generamos df1 concatenando ahoranoticiasmega, horas24 y elciudadano
medios = [dataset.loc[dataset['media_outlet']=="ahoranoticiasmega"],
          dataset.loc[dataset['media_outlet']=="horas24"],
          dataset.loc[dataset['media_outlet']=="elciudadano"],
          dataset.loc[dataset['media_outlet']=="latercera"]]
df1 = pd.concat(medios)
for index, row in df1.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url) 
    topic=obj[0][2]
    df1.loc[index,'topic'] = topic.lower()

Las urls de _Biobiochile, El Mostrador, Emol_ poseen dos columnas de tópicos. La primera columna entrega información irrelevante. 

In [6]:
medios2 = [dataset.loc[dataset['media_outlet']=="biobiochile"],
           dataset.loc[dataset['media_outlet']=="elmostrador"],
           dataset.loc[dataset['media_outlet']=="emol"]] 
df2 = pd.concat(medios2)
for index, row in df2.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url)
    topic = obj[0][3]
    df2.loc[index,'topic'] = topic.lower()

In [7]:
df = pd.concat([df1,df2])
np.sort(df['topic'].unique())

array(['2020', '2021', 'actualidad', 'animal', 'artes', 'autos',
       'calidad-de-vida', 'cartas-ciudadanas', 'chile',
       'ciencia-tecnologia', 'ciudadanos-al-poder', 'columnas',
       'conversacioneslt', 'coronavirus', 'culto', 'data', 'deportes',
       'derechos-humanos', 'destacados-cultura', 'economia', 'educacion',
       'el-deportivo', 'especiales', 'espectaculos', 'genero',
       'internacional', 'justicia', 'la-tercera-domingo', 'la-tercera-pm',
       'la-tercera-tv', 'latinoamerica', 'mapuche', 'masdeco',
       'medio-ambiente', 'mexico', 'mineria', 'mouse', 'multimedia',
       'mundo', 'nacional', 'noticiasbbc', 'opinion', 'pais', 'paula',
       'peru', 'plebiscito-chile-elige', 'politica', 'portada',
       'portada-tv', 'proceso-constituyente', 'programas', 'pueblos',
       'pulso', 'pulso-trader', 'que-pasa', 'reconstitucion',
       'redes-sociales', 'regiones', 'reportaje-investigacion',
       'revista-que-pasa', 'salud', 'sociedad', 'tecnologia',
       

In [8]:
df['label'] = ""
# mundo
df.loc[df['topic'] == 'mundo',                  'label'] = 0
df.loc[df['topic'] == 'noticiasbbc',            'label'] = 0
df.loc[df['topic'] == 'internacional',          'label'] = 0
# economia
df.loc[df['topic'] == 'economia',               'label'] = 1
df.loc[df['topic'] == 'pulso-trader',           'label'] = 1
# política y conflictos
df.loc[df['topic'] == 'politica',               'label'] = 2
df.loc[df['topic'] == 'plebiscito-chile-elige', 'label'] = 2
df.loc[df['topic'] == 'ciudadanos-al-poder',    'label'] = 2
df.loc[df['topic'] == 'proceso-constituyente',  'label'] = 2
# ciencia y tecnologia
df.loc[df['topic'] == 'tecnologia',             'label'] = 3
df.loc[df['topic'] == 'tecnologia-2',           'label'] = 3
df.loc[df['topic'] == 'ciencia-tecnologia',     'label'] = 3
df.loc[df['topic'] == 'mouse',                  'label'] = 3
df.loc[df['topic'] == 'redes-sociales',         'label'] = 3
# accidentes 
df.loc[df['topic'] == 'accidentes',             'label'] = 4
# arte y cultura
df.loc[df['topic'] == 'cultura',                'label'] = 5
df.loc[df['topic'] == 'artes',                  'label'] = 5
# deportes
df.loc[df['topic'] == 'deportes',               'label'] = 6
df.loc[df['topic'] == 'el-deportivo',           'label'] = 6
# ecologia y planeta
df.loc[df['topic'] == 'medio-ambiente',         'label'] = 7
df.loc[df['topic'] == 'animal',                 'label'] = 7
# crimen, delitos y justicia
df.loc[df['topic'] == 'justicia',               'label'] = 8
df.loc[df['topic'] == 'reportaje-investigacion','label'] = 8
# salud
df.loc[df['topic'] == 'salud',                  'label'] = 9
df.loc[df['topic'] == 'calidad-de-vida',        'label'] = 9
df.loc[df['topic'] == 'coronavirus',            'label'] = 9

display(df['label'].value_counts())
# nos quedamos sólo con las noticias etiquetadas. para clasificar df1 utilizaremos otro metodo
df1 = df.loc[df['label']==''] 
df = df[df['label']!='']



     34315
0     9913
6     9609
3     5689
2     5020
1     1774
8      428
9      244
5      179
7      168
Name: label, dtype: int64

Tenemos muchas noticias no clasificadas, por lo que es necesario utilizar otro mecanísmo para clasificar las noticias restantes.

### 3.  Extracción de términos clave de columna title+text <a name="b"></a>

In [9]:
matcher = Matcher(nlp.vocab)

# Pattern 1: NOUN de NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# Pattern 2: NOUN ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

# Pattern 3: 
pattern_3 = [{"POS": "NOUN"}]
matcher.add("NOUN", [pattern_3])

    Bloque de alto coste computacional ⬇️

In [10]:
df1['terms'] = ""
for index, row in df1.iterrows():
    doc = nlp(row['content'].lower())

    matches = matcher(doc)
    categories = ""
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        categories = categories + span.text + "; "
    
    row['terms'] = categories
    
display(df1.head(3))

,media_outlet,url,text,content,topic,label,terms
268,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312353-si...,"La noche de este martes 01 de septiembre, a la...",Temblor de menor intensidad afecta nuevamente ...,nacional,,temblor; intensidad; noche; martes; septiembre...
269,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312350-cy...,"En la presente edición del CyberDay, el rubro ...",CyberDay 2020: Consolas y videojuegos son los ...,nacional,,cyberday; 2020; consolas; videojuegos; product...
270,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312347-on...,La Oficina Nacional de Emergencia del Minister...,Sismos en el norte: Expertos aclaran que exist...,nacional,,norte; expertos; energía; energía acumulada; z...


In [12]:
from collections import Counter
df1["most_common"] = ""
df1["second_common"] = ""

for index, row in df1.iterrows():
    split_it = row['terms'].split("; ")
    counter = Counter(split_it)
    row['most_common'] = counter.most_common()[0][0]
    row['second_common'] = counter.most_common()[1][0]

display(df1.head(3))
display(df1['most_common'].value_counts())
display(df1['second_common'].value_counts())


,media_outlet,url,text,content,topic,label,terms,most_common,second_common,third_common
268,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312353-si...,"La noche de este martes 01 de septiembre, a la...",Temblor de menor intensidad afecta nuevamente ...,nacional,,temblor; intensidad; noche; martes; septiembre...,intensidad,sismo,región
269,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312350-cy...,"En la presente edición del CyberDay, el rubro ...",CyberDay 2020: Consolas y videojuegos son los ...,nacional,,cyberday; 2020; consolas; videojuegos; product...,consolas,precios,cyberday
270,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312347-on...,La Oficina Nacional de Emergencia del Minister...,Sismos en el norte: Expertos aclaran que exist...,nacional,,norte; expertos; energía; energía acumulada; z...,réplicas,sismo,magnitud


casos          889
personas       654
shared         627
carabineros    562
salud          460
              ... 
phishing         1
cadáver          1
veredas          1
críticos         1
motel            1
Name: most_common, Length: 4292, dtype: int64

personas                   514
salud                      429
años                       365
casos                      346
región                     310
                          ... 
firmeza                      1
subsecretario de redes       1
condenados                   1
cifra de hospitalizados      1
caminos                      1
Name: second_common, Length: 5812, dtype: int64

In [13]:
df1.loc[df1['most_common'] == 'camioneros',             'label'] = 8
df1.loc[df1['most_common'] == 'acusación',              'label'] = 8
df1.loc[df1['most_common'] == 'casos',                  'label'] = 9
df1.loc[df1['most_common'] == 'ley',                    'label'] = 8


df1.loc[df1['second_common'] == 'gobierno'              'label'] = 2
df1.loc[df1['second_common'] == 'paro'                  'label'] = 2


# agregar mas...

df1['label'].value_counts()

     33127
9      889
8      299
Name: label, dtype: int64

In [14]:
df = pd.concat([df,df1])
df.drop(columns=['most_common','second_common','third_common','terms'], inplace=True)
df['label'].value_counts()

     33127
0     9913
6     9609
3     5689
2     5020
1     1774
9     1133
8      727
5      179
7      168
Name: label, dtype: int64

In [17]:
df.to_csv("topics.csv")

### 5.  Balance de clases dataset de entrenamiento <a name="d"></a>
Visualizamos la densidad de los tópicos con tal de combatir el sobreajuste si corresponde

In [16]:
#fig, ax = plt.subplots(1, figsize=(9,5), tight_layout=True)
#ax.hist(dataset['category'],edgecolor = 'black',bins=dataset['category'].unique())
#plt.xticks(rotation='vertical', fontsize=7)
#plt.ylabel("densidad")
#plt.show()